# Un interprete del $\lambda$-calcolo

## La sintassi del $\lambda$-calcolo

$$
e ::= x \mid \lambda x.e \mid e \, e
$$

Definiamo il tipo degli identificatori e dell'AST delle espressioni:

In [1]:
type id = string

type exp =
    Var of id
  | Lam of id * exp
  | App of exp * exp

type id = string


type exp = Var of id | Lam of id * exp | App of exp * exp


## La semantica del $\lambda$-calcolo

$$
(\lambda x . e_1) e_2 \rightarrow e_1 \{ x := e_2 \}
$$

$$
\frac{e_1 \rightarrow e_1'}{e_1 \, e_2 \rightarrow e_1' \, e_2}
\qquad 
\frac{e_2 \rightarrow e_2'}{e_1 \, e_2 \rightarrow e_1 \, e_2'}
$$

$$
\frac{e \rightarrow e'}{\lambda x . e \rightarrow \lambda x . e'}
$$

## Implementazione della capture-avoiding substitution 

Richiamo della definizione:

$$
\begin{array}{rcl}
x \{ x:= e \} & \equiv & e\\
y \{ x:= e \} & \equiv & y \mbox{ se } y \neq x\\
(e_1 e_2) \{ x:= e \} & \equiv & ( e_1 \{ x:= e \} ) ( e_2 \{ x:= e \} )\\
(\lambda y . e_1) \{ x:= e \} & \equiv & \lambda y . (e_1 \{ x:= e \}) \\
& & \mbox{ se } y \neq x \mbox{ e } y \not\in FV(e)\\
(\lambda y . e_1) \{ x:= e \} & \equiv & \lambda z . ((e_1 \{ y:= z \}) \{ x:= e \})\\
& & \mbox{ se } y \neq x \mbox{ e } y \in FV(e) \mbox{ e } z \mbox{ fresca}
\end{array}
$$



## Implementazione di FV(e)

$$
\small
FV(x) = x \qquad FV(\lambda x.e) = FV(e) \setminus \{x\}
$$

$$
\small
FV(e_1\,e_2) = FV(e_1)\cup FV(e_2)
$$

Funzione che calcola ricorsivamente l'insieme delle *variabili libere* dell'espressione $e$ rappresentato come lista di identificatori:

In [2]:
(* computes the free (non-bound) variables in e *)
let rec fvs e =
  match e with
      Var x -> [x]
    | Lam (x,e) -> List.filter (fun y -> x <> y) (fvs e)
    | App (e1,e2) -> (fvs e1) @ (fvs e2)
;;

val fvs : exp -> id list = <fun>


Esempi d'uso:

In [3]:
(* TESTS *)
fvs (Var "x") = ["x"];;
fvs (Lam ("x",Var "y")) = ["y"];;
fvs (Lam ("x",Var "x")) = [];;
fvs (App (Lam ("x", Var "z"), Var "y")) = ["z"; "y"];;

- : bool = true


- : bool = true


- : bool = true


- : bool = true


## Generatore di identificatori "freschi"

La capture avoiding substitution richiede in alcuni casi di generare identificatori *freschi* (nuovi, non presenti nell'espressione)

In [4]:
(* generates a fresh variable   *)
let newvar =
  let x = ref 0 in
  fun () -> 
    let c = !x in
    incr x;
    "v"^(string_of_int c)

val newvar : unit -> string = <fun>


`newvar` è una funzione senza parametri con una *variabile modificabile* `x` nella sua *chiusura*!

Esempi d'uso:

In [5]:
(* TESTS *)
newvar ();;
newvar ();;

- : string = "v0"


- : string = "v1"


Ora siamo pronti per implementare la capture-avoiding substitution:

In [6]:
(* substitution: subst e y m means 
  "substitute occurrences of variable y with m in the expression e" *)
let rec subst e y m =
  match e with
      Var x -> 
    if y = x then m (* replace x with m *)
    else e (* variables don't match: leave x unchanged *)
    | App (e1,e2) -> App (subst e1 y m, subst e2 y m)
    | Lam (x,e) ->
      if y = x then (* don't substitute under the variable binder *)
        Lam(x,e)
      else if not (List.mem x (fvs m)) then (* no need to alpha convert *)
        Lam (x, subst e y m)
      else (* need to alpha convert *)
        let z = newvar() in (* assumed to be "fresh" *)
        let e' = subst e x (Var z) in (* replace x with z in e *)
        Lam (z,subst e' y m) (* substitute for y in the adjusted term, e' *)

val subst : exp -> id -> exp -> exp = <fun>


In [7]:
(* TESTS *)
let m1 =  (App (Var "x", Var "y"));;  (* x y *)
let m2 = (App (Lam ("z",Var "z"), Var "w"));; (* (lambda z . z) w *)
let m3 = (App (Lam ("z",Var "x"), Var "w"));; (* (lambda z . x) w *)
let m4 = (App (App (Lam ("z",Var "z"), Lam ("x", Var "x")), Var "w"))
            (* (lambda z . z) (lambda x . x) w *)
            
let m1_zforx = subst m1 "x" (Var "z");;
let m1_m2fory = subst m1 "y" m2
let m2_ughforz = subst m2 "z" (Var "ugh")
let m3_zforx = subst m3 "x" (Var "z")
let m1_m3fory = subst m1 "y" m3

val m1 : exp = App (Var "x", Var "y")


val m2 : exp = App (Lam ("z", Var "z"), Var "w")


val m3 : exp = App (Lam ("z", Var "x"), Var "w")


val m4 : exp = App (App (Lam ("z", Var "z"), Lam ("x", Var "x")), Var "w")


val m1_zforx : exp = App (Var "z", Var "y")


val m1_m2fory : exp = App (Var "x", App (Lam ("z", Var "z"), Var "w"))


val m2_ughforz : exp = App (Lam ("z", Var "z"), Var "w")


val m3_zforx : exp = App (Lam ("v2", Var "z"), Var "w")


val m1_m3fory : exp = App (Var "x", App (Lam ("z", Var "x"), Var "w"))


## Implementazione della semantica

$$
\small
(\lambda x . e_1) e_2 \rightarrow e_1 \{ x := e_2 \}
$$


$$
\small
\frac{e_1 \rightarrow e_1'}{e_1 \, e_2 \rightarrow e_1' \, e_2}
\quad 
\frac{e_2 \rightarrow e_2'}{e_1 \, e_2 \rightarrow e_1 \, e_2'}
\quad
\frac{e \rightarrow e'}{\lambda x . e \rightarrow \lambda x . e'}
$$

In [8]:
(* beta reduction. *)
let rec reduce e =
  match e with
      App (Lam (x,e1), e2) -> subst e1 x e2 (* direct beta rule *)
    | App (e1,e2) -> 
      let e1' = reduce e1 in (* try to reduce a term in the lhs *)
      if e1'<>e1 then App(e1',e2)
      else App (e1,reduce e2) (* didn't work; try rhs *)
    | Lam (x,e) -> Lam (x, reduce e) (* reduce under the lambda (!) *)
    | _ -> e (* no opportunity to reduce *)

val reduce : exp -> exp = <fun>


In [9]:
(* TESTS *)
let m2red = reduce m2 ;; (* (lambda z . z) w *)
let m3red = reduce m3 ;; (* (lambda z . x) w *)
let m4red1 = reduce m4 ;; (* (lambda z . z) (lambda x . x) w *)
let m4red2 = reduce m4red1 ;; (* vedi sopra *)
let m13sred = reduce m1_m3fory ;; (* x ((lambda z . x) w) *)

val m2red : exp = Var "w"


val m3red : exp = Var "x"


val m4red1 : exp = App (Lam ("x", Var "x"), Var "w")


val m4red2 : exp = Var "w"


val m13sred : exp = App (Var "x", Var "x")


*DOMANDA:* Ma la semantica che abbiamo implementato è effettivamente questa?

$$
(\lambda x . e_1) e_2 \rightarrow e_1 \{ x := e_2 \}
$$


$$
\frac{e_1 \rightarrow e_1'}{e_1 \, e_2 \rightarrow e_1' \, e_2}
\qquad 
\frac{e_2 \rightarrow e_2'}{e_1 \, e_2 \rightarrow e_1 \, e_2'}
$$

$$
\frac{e \rightarrow e'}{\lambda x . e \rightarrow \lambda x . e'}
$$

*Non esattamente...*

* L'*ordine dei pattern* nel pattern matching *privilegia l'applicazione funzionale* alla riduzione di $e_1$ e di $e_2$

```
match e with
      App (Lam (x,e1), e2) -> subst e1 x e2 (* direct beta rule *)
    | App (e1,e2) -> ... 
```

* Inoltre, questo modo di gestire l'applicazione funzionale dice che `e2` può essere ridotto *solo se `e1` non è riducibile*:

```
App (e1,e2) -> 
      let e1' = reduce e1 in (* try to reduce a term in the lhs *)
      if e1'<>e1 then App(e1',e2)
      else App (e1,reduce e2) (* didn't work; try rhs *)
```



Quindi la semantica che abbiamo implementato in realtà è questa:

$$
(\lambda x . e_1) e_2 \rightarrow e_1 \{ x := e_2 \}
$$

$$
\frac{e_1 \neq_{\alpha} \lambda x . e_3 \quad e_1 \rightarrow e_1'}{e_1 \, e_2 \rightarrow e_1' \, e_2}
\qquad 
\frac{e_1 \neq_{\alpha} \lambda x . e_3 \quad e_1 \not\rightarrow \quad e_2 \rightarrow e_2'}{e_1 \, e_2 \rightarrow e_1 \, e_2'}
$$

$$
\frac{e \rightarrow e'}{\lambda x . e \rightarrow \lambda x . e'}
$$

dove:

* $=_{\alpha}$ è l'$\alpha$-equivalenza
* $e_1 \not\rightarrow$ significa $\not\exists e_1' . e_1 \rightarrow e_1'$. 

D'altra parte la definizione originale è *non deterministica*

* la stessa espressione può fare due riduzioni diverse

Esempio:

$$
((\lambda x . x) \, y)\,((\lambda x . x)\, y) \rightarrow y \, ((\lambda x . x) \, y)
$$
$$
((\lambda x . x) \, y)\,((\lambda x . x)\, y) \rightarrow ((\lambda x . x) \, y) \, y
$$

Invece *l'interprete è un programma* e la sua esecuzione deve essere *deterministica*

* si può dimostrare, sfruttando la proprietà di confluenza del $\lambda$-calcolo (i.e., Church-Rosser) che nonostante le condizioni aggiunte non si perdono computazioni possibili

## PER ESERCIZIO...

* Implementare la *chiusura transitiva* della semantica, che consente di eseguire intere computazioni invece che singoli passi. Che succede se le diamo in pasto l'espressione $\Omega = (\lambda x . x x)(\lambda x . x x)$ ?
* Implementare la semantica *call-by-value* del $\lambda$-calcolo
* Implementare la seguente *semantica big-step* del $\lambda$-calcolo e verificare che succedere dandole in pasto l'espressione $\Omega$:

$$
    \frac{e \rightarrow_{bs} e'}{\lambda x.e \rightarrow_{bs} \lambda x.e'}
\qquad
    x \rightarrow_{bs} x
$$

$$
    \frac{e_1 \rightarrow_{bs} \lambda x.e_1' \quad e_2 \rightarrow_{bs} e_2' \quad e_1'\{x:=e_2'\} \rightarrow_{bs} e'}{e_1 \, e_2 \rightarrow_{bs} e'}
$$



## Funzioni di utilità per la stampa delle $\lambda$-espressioni

In [10]:
(* pretty printing *)
open Format;;

let ident = print_string;;
let kwd = print_string;;

let rec print_exp0 = function
  | Var s ->  ident s
  | lam -> open_hovbox 1; kwd "("; print_lambda lam; kwd ")"; close_box ()

and print_app = function
  | e -> open_hovbox 2; print_other_applications e; close_box ()

and print_other_applications f =
  match f with
  | App (f, arg) -> print_app f; print_space (); print_exp0 arg
  | f -> print_exp0 f

and print_lambda = function
  | Lam (s, lam) ->
      open_hovbox 1;
      kwd "\\"; ident s; kwd "."; print_space(); print_lambda lam;
      close_box()
  | e -> print_app e;;

val ident : string -> unit = <fun>


val kwd : string -> unit = <fun>


val print_exp0 : exp -> unit = <fun>
val print_app : exp -> unit = <fun>
val print_other_applications : exp -> unit = <fun>
val print_lambda : exp -> unit = <fun>


In [11]:
(* TESTS *)
print_lambda m1; print_newline ();;
print_lambda m2; print_newline ();;

x y


- : unit = ()


(\z. z) w


- : unit = ()
